Implementation Plan:

You can run git commands like this to get the latest version of code to work with:

git checkout preprod && git pull && git checkout -B uc-code-refactor-20230225 && git checkout uc-code-refactor-20230224 ./uc1Codefix.ipynb

In [1]:
import os, re, fnmatch

In [4]:
def get_strings_to_replace(full_path, pattern):
    strings_to_replace = []
    for filename in os.listdir(full_path):
        if filename.endswith(".py"):
            with open(os.path.join(full_path, filename), "r") as f:
                for line in f:
                    for word in line.split():
                        if pattern.search(word): 
                            if (len(word.split("."))) == 2:
                                strings_to_replace.append( re.sub( r'[)"](.*)', '', (re.sub('[^A-Za-z0-9.]+_', '', word)) ) )
                            else:
                                print (f"This was formatted poorly and is hence not included in the list:\n \'{filename} \'- {word}\n")
                                
    return strings_to_replace

**Facts Notebooks Refactoring**

Adding additional imports to the top of the notebook

In [3]:
directory   = "../transform/facts/"

In [ ]:
find_replace(directory, '', '# MAGIC %run ../../Common/common-transform', "# MAGIC %run ../../common/common-transform \n\n# COMMAND ---------- \n\n# MAGIC %run ../../common/common-helpers \n# COMMAND ---------- \n")

Removing hardcoded hive metastore references.

In [ ]:
findstr     = 'hive_metastore.'
replacestr  = ''
find_replace(directory, '', findstr, replacestr)

Adding f string where get_table_name function was called in place of hardcoded table names

In [ ]:
find_replace(directory, '', 'spark.sql("""', 'spark.sql(f"""')

Finding and replacing the hive mestastore table namespaces using regex pattern.

In [ ]:
directory = "../Transform/Dimension/"

In [ ]:
patternstr = re.compile(r"\braw.\b")
strings_to_replace =  get_strings_to_replace(full_path = directory, pattern = patternstr)


for findstr in sorted(strings_to_replace, key = len, reverse = True):
    layer   = findstr.split(".")[0]
    schema  = findstr.split(".")[1].split("_")[0]
    table   = findstr.split(".")[1]
    replacestr  = f'{{get_table_namespace(\'{layer}\', \'{table}\')}}'
    find_replace(directory, '', findstr, replacestr)

In [ ]:
#repeat above pattern for other layers. I'd tried converting above to a function but had an issue with the regex. need to revisit.
patternstr = re.compile(r"\bcleansed.\b")
strings_to_replace =  get_strings_to_replace(full_path = directory, pattern = patternstr)


for findstr in sorted(strings_to_replace, key = len, reverse = True):
    layer   = findstr.split(".")[0]
    schema  = findstr.split(".")[1].split("_")[0]
    table   = findstr.split(".")[1]
    replacestr  = f'{{get_table_namespace(\'{layer}\', \'{table}\')}}'
    find_replace(directory, '', findstr, replacestr)

In [ ]:
patternstr = re.compile(r"\bcurated.\b")
strings_to_replace =  get_strings_to_replace(full_path = directory, pattern = patternstr)


for findstr in sorted(strings_to_replace, key = len, reverse = True):
    layer   = findstr.split(".")[0]
    schema  = findstr.split(".")[1].split("_")[0]
    table   = findstr.split(".")[1]
    replacestr  = f'{{get_table_namespace(\'{layer}\', \'{table}\')}}'
    find_replace(directory, '', findstr, replacestr)

In instances where the raw, cleansed and curated aren't explicitly called in the namespace, you can use adhoc find and replace as below.

In [ ]:
#in this instance, the codebase has soft coded tablenames like '{SOURCE}.tablename'
findstr     = '{SOURCE}.tablename'
replacestr  = '{get_table_namespace(f\'{SOURCE}\', \'tablename\')}'
find_replace(directory, '', findstr, replacestr)

Below instances show a variety of conventions currently used in codebase which need to be replaced by the new approach.

In [ ]:
findstr     = '{TARGET}.tablename'
replacestr  = '{get_table_namespace(f\'{TARGET}\', \'tablename\')}'
find_replace(directory, '', findstr, replacestr)

In [ ]:

findstr     = 'GetTable(f"{TARGET}.tablename")'
replacestr  = 'GetTable(f"{get_table_namespace(f\'{TARGET}\', \'tablename\')}")'
find_replace(directory, '', findstr, replacestr)